In [4]:
import os
import numpy as np
import tifffile
from cellpose import train, models

BASE_DIR = ".."
TRAINING_DIR = os.path.join(BASE_DIR, "data", "training")
MODELS_DIR = os.path.join(BASE_DIR, "models")

os.makedirs(MODELS_DIR, exist_ok=True)

print("\n--- FINE-TUNING 'cpsam' MODEL ON DENOISED IMAGES ---")
img_dir = os.path.join(TRAINING_DIR, "images", "nucleus_binary")
lbl_dir = os.path.join(TRAINING_DIR, "labels", "nucleus")

train_files = []
train_labels_files = []

print("Finding and matching training image-label pairs...")
potential_image_files = sorted([f for f in os.listdir(img_dir) if f.endswith('_projection.tif')])

for filename in potential_image_files:
    label_path = os.path.join(lbl_dir, filename)
    if os.path.exists(label_path):
        train_files.append(os.path.join(img_dir, filename))
        train_labels_files.append(label_path)
    else:
        print(f"Warning: Found image '{filename}' but no corresponding label. Skipping.")

if not train_files:
    print("ERROR: Could not find any matched image-label pairs. Please check your training folders.")
else:
    print(f"Found {len(train_files)} matched image-label pairs for training.")

    model = models.CellposeModel(gpu=True, model_type='cyto2')
    model_name = "nucleus_cell_model_cpsam_v3"

    print(f"\nTraining {model_name}...")
    train.train_seg(
        model.net,
        train_files=train_files,
        train_labels_files=train_labels_files,
        save_path=MODELS_DIR,
        model_name=model_name,
        n_epochs=400,
        learning_rate=0.005,
        batch_size=4,
        weight_decay=0.0001,
        min_train_masks=1
    )
    print(f"--- 'cpsam' model fine-tuning complete. ---")

model_type argument is not used in v4.0.1+. Ignoring this argument...



--- FINE-TUNING 'cpsam' MODEL ON DENOISED IMAGES ---
Finding and matching training image-label pairs...
Found 16 matched image-label pairs for training.

Training nucleus_cell_model_cpsam_v3...


100%|██████████| 16/16 [00:00<00:00, 2326.45it/s]


--- 'cpsam' model fine-tuning complete. ---
